NCR Ride Bookings Preprocessing

In [1]:
pip install mlflow optuna joblib lightgbm shap

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error
import mlflow
import mlflow.lightgbm
import mlflow.xgboost
import optuna
import joblib
import logging
import shap

In [3]:
# ---------------------------- CONFIG ----------------------------------
CONFIG = {
    "ride_data_path": "ncr_ride_bookings.csv",
    "weather_data_path": "kaggel_weather_2013_to_2024.csv",
    "surge_threshold_quantile": 0.75,
    "random_state": 42,
    "test_size": 0.2
}

logging.basicConfig(level=logging.INFO, format='[%(levelname)s] %(message)s')
logger = logging.getLogger(__name__)

In [4]:
# ---------------------------- DATA PREPROCESSING -----------------------
def preprocess_ride_data(path):
    logger.info("Loading and preprocessing ride data...")
    df = pd.read_csv(path)
    df.drop(["Booking ID", "Customer ID"], axis=1, inplace=True)

    df['Original_Date'] = pd.to_datetime(df['Date'])
    df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
    df['Hour'] = df['Datetime'].dt.hour
    df['Day'] = df['Datetime'].dt.day
    df['Month'] = df['Datetime'].dt.month
    df['Weekday'] = df['Datetime'].dt.dayofweek
    df['Is_Weekend'] = df['Weekday'].isin([5, 6]).astype(int)
    df.drop(['Date', 'Time', 'Datetime'], axis=1, inplace=True)

    for col in ['Avg VTAT', 'Avg CTAT', 'Booking Value', 'Ride Distance']:
        df[col] = df.groupby('Vehicle Type')[col].transform(lambda x: x.fillna(x.median()))
    for col in ['Driver Ratings', 'Customer Rating']:
        df[col] = df.groupby('Vehicle Type')[col].transform(lambda x: x.fillna(x.mean()))

    df['Cancelled Rides by Customer'] = df['Cancelled Rides by Customer'].fillna(0)
    df['Cancelled Rides by Driver'] = df['Cancelled Rides by Driver'].fillna(0)
    df['Incomplete Rides'] = df['Incomplete Rides'].fillna(0)
    df['Reason for cancelling by Customer'] = df['Reason for cancelling by Customer'].fillna("No Cancellation")
    df['Driver Cancellation Reason'] = df['Driver Cancellation Reason'].fillna("No Cancellation")
    df['Incomplete Rides Reason'] = df['Incomplete Rides Reason'].fillna("Completed")
    df['Payment Method'] = df['Payment Method'].fillna("Unknown")

    fare_threshold = df['Booking Value'].quantile(CONFIG["surge_threshold_quantile"])
    df['Is_Surge'] = (df['Booking Value'] > fare_threshold).astype(int)

    le = LabelEncoder()
    string_cat_cols = ['Vehicle Type', 'Pickup Location', 'Drop Location', 'Payment Method']
    for col in string_cat_cols:
        df[col + '_Encoded'] = le.fit_transform(df[col])
        df.drop(col, axis=1, inplace=True)

    other_cat_cols = ['Reason for cancelling by Customer', 'Driver Cancellation Reason',
                      'Incomplete Rides Reason', 'Booking Status', 'Is_Surge']
    for col in other_cat_cols:
        df[col + '_Encoded'] = le.fit_transform(df[col].astype(str))
        df.drop(col, axis=1, inplace=True)

    logger.info(f"Ride data preprocessing complete. Shape: {df.shape}")
    return df

In [5]:
def preprocess_weather_data(path):
    logger.info("Loading and cleaning weather data...")
    weather_df = pd.read_csv(path)
    if 'Unnamed: 0' in weather_df.columns:
        weather_df.drop(columns=['Unnamed: 0'], inplace=True)
    weather_df['DATE'] = pd.to_datetime(weather_df['DATE'])
    weather_df.fillna(method='ffill', inplace=True)
    weather_df.columns = weather_df.columns.str.strip().str.lower().str.replace(' ', '_')
    logger.info(f"Weather data cleaned. Shape: {weather_df.shape}")
    return weather_df

In [6]:
def merge_and_scale(df, weather_df):
    logger.info("Merging ride and weather data...")
    weather_2024 = weather_df[weather_df['year'] == 2024].copy()
    merged_df = pd.merge(df, weather_2024, left_on='Original_Date', right_on='date', how='left')
    merged_df.drop(columns=['date'], inplace=True)

    weather_columns = [c for c in weather_df.columns if c not in ['date']]
    merged_df.sort_values(by='Original_Date', inplace=True)
    merged_df[weather_columns] = merged_df[weather_columns].ffill()

    num_cols = merged_df.select_dtypes(include=['int64', 'float64']).columns.tolist()
    exclude_from_scaling = ['year', 'month_y', 'day_y']
    num_cols = [col for col in num_cols if col not in exclude_from_scaling]

    scaler = StandardScaler()
    merged_df[num_cols] = scaler.fit_transform(merged_df[num_cols])

    joblib.dump(scaler, "scaler.pkl")
    logger.info("Scaling complete and scaler saved as scaler.pkl")

    merged_df.to_csv('preprocessed_ride_data.csv', index=False)
    logger.info(f"Merged preprocessed data saved. Shape: {merged_df.shape}")
    return merged_df


In [7]:
# ---------------------------- MODEL TRAINING ---------------------------
def train_lightgbm(df):
    logger.info("Training LightGBM model...")

    X = df.drop(['Booking Value', 'Original_Date'], axis=1)
    y = df['Booking Value']

    # Handle any object columns that might have come from weather (like 'conditions')
    object_cols = X.select_dtypes(include=['object']).columns
    if len(object_cols) > 0:
        logger.warning(f"Encoding object columns: {list(object_cols)}")
        for col in object_cols:
            X[col] = X[col].astype('category').cat.codes

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=CONFIG["test_size"], random_state=CONFIG["random_state"]
    )

    def objective(trial):
        params = {
            'objective': 'regression_l1',
            'metric': 'rmse',
            'n_estimators': trial.suggest_int('n_estimators', 500, 2000),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
            'num_leaves': trial.suggest_int('num_leaves', 20, 300),
            'max_depth': trial.suggest_int('max_depth', 3, 12),
            'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'random_state': CONFIG["random_state"],
            'n_jobs': -1,
        }
        model = lgb.LGBMRegressor(**params)
        model.fit(X_train, y_train, eval_set=[(X_test, y_test)],
                  callbacks=[lgb.early_stopping(100, verbose=False)])
        preds = model.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, preds))
        return rmse

    # --------------------- Hyperparameter Tuning ---------------------
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=50)
    best_params = study.best_params

    # --------------------- Model Training ---------------------
    mlflow.set_experiment("Ride Surge Prediction")
    with mlflow.start_run(run_name="Tuned LightGBM"):
        mlflow.log_params(best_params)

        model = lgb.LGBMRegressor(**best_params, random_state=CONFIG["random_state"])
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, preds))
        mlflow.log_metric("rmse", rmse)
        mlflow.lightgbm.log_model(model, "lgbm-ride-surge-model")

        # --------------------- Explainable AI (SHAP) ---------------------
        logger.info("Generating SHAP explainability plots...")
        import shap
        import matplotlib.pyplot as plt

        # Create SHAP explainer
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(X_test)

        # Global Feature Importance Plot
        shap.summary_plot(shap_values, X_test, plot_type="bar", show=False)
        plt.title("Feature Importance (SHAP)")
        plt.tight_layout()
        plt.savefig("shap_summary_bar.png", dpi=200)
        plt.close()

        # Detailed Beeswarm Plot
        shap.summary_plot(shap_values, X_test, show=False)
        plt.title("SHAP Feature Impact (Beeswarm)")
        plt.tight_layout()
        plt.savefig("shap_summary_beeswarm.png", dpi=200)
        plt.close()

        # Log SHAP plots in MLflow
        mlflow.log_artifact("shap_summary_bar.png")
        mlflow.log_artifact("shap_summary_beeswarm.png")

        logger.info("SHAP explainability plots generated and logged to MLflow.")

    # --------------------- Save Model ---------------------
    joblib.dump(model, "lgbm_model.pkl")
    logger.info(f"LightGBM training complete. RMSE: {rmse:.3f}. Model saved as lgbm_model.pkl")

    return model, rmse

In [8]:
def train_xgboost(df):
    logger.info("Training XGBoost model...")

    X = df.drop(['Booking Value', 'Original_Date'], axis=1)
    y = df['Booking Value']

    # Handle any object columns that might have come from weather (like 'conditions')
    object_cols = X.select_dtypes(include=['object']).columns
    if len(object_cols) > 0:
        logger.warning(f"Encoding object columns: {list(object_cols)}")
        for col in object_cols:
            X[col] = X[col].astype('category').cat.codes

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=CONFIG["test_size"], random_state=CONFIG["random_state"]
    )

    # --------------------- Hyperparameter Tuning ---------------------
    def objective_xgb(trial):
        params = {
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'n_estimators': trial.suggest_int('n_estimators', 500, 2500),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
            'max_depth': trial.suggest_int('max_depth', 3, 12),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'gamma': trial.suggest_float('gamma', 0, 0.5),
            'random_state': CONFIG["random_state"],
            'n_jobs': -1,
            'tree_method': 'hist',
            'enable_categorical': True
        }
        model = xgb.XGBRegressor(**params)
        model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)
        preds = model.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, preds))
        return rmse

    study_xgb = optuna.create_study(direction='minimize')
    study_xgb.optimize(objective_xgb, n_trials=50)
    best_params = study_xgb.best_params

    # --------------------- Model Training ---------------------
    mlflow.set_experiment("Ride Surge Prediction")
    with mlflow.start_run(run_name="Tuned XGBoost"):
        mlflow.log_params(best_params)

        model = xgb.XGBRegressor(**best_params, random_state=CONFIG["random_state"],
                                 tree_method='hist', enable_categorical=True)
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, preds))
        mlflow.log_metric("rmse", rmse)
        mlflow.xgboost.log_model(model, "xgb-ride-surge-model")

        # --------------------- Explainable AI (SHAP) ---------------------
        logger.info("Generating SHAP explainability plots for XGBoost...")
        import shap
        import matplotlib.pyplot as plt

        # Create SHAP explainer (TreeExplainer is best for XGBoost)
        explainer = shap.Explainer(model)
        shap_values = explainer(X_test)

        # Global Feature Importance (bar plot)
        shap.summary_plot(shap_values, X_test, plot_type="bar", show=False)
        plt.title("XGBoost Feature Importance (SHAP)")
        plt.tight_layout()
        plt.savefig("xgb_shap_summary_bar.png", dpi=200)
        plt.close()

        # Beeswarm (detailed impact)
        shap.summary_plot(shap_values, X_test, show=False)
        plt.title("XGBoost SHAP Feature Impact (Beeswarm)")
        plt.tight_layout()
        plt.savefig("xgb_shap_summary_beeswarm.png", dpi=200)
        plt.close()

        # Log plots in MLflow
        mlflow.log_artifact("xgb_shap_summary_bar.png")
        mlflow.log_artifact("xgb_shap_summary_beeswarm.png")

        logger.info("SHAP explainability plots for XGBoost saved and logged in MLflow.")

    # --------------------- Save Model ---------------------
    joblib.dump(model, "xgb_model.pkl")
    logger.info(f"XGBoost training complete. RMSE: {rmse:.3f}. Model saved as xgb_model.pkl")

    return model, rmse

In [9]:
# ---------------------------- MAIN ------------------------------------
if __name__ == "__main__":
    ride_df = preprocess_ride_data(CONFIG["ride_data_path"])
    weather_df = preprocess_weather_data(CONFIG["weather_data_path"])
    merged_df = merge_and_scale(ride_df, weather_df)

    lgb_model, lgb_rmse = train_lightgbm(merged_df)
    xgb_model, xgb_rmse = train_xgboost(merged_df)

    logger.info(f"✅ All training complete. LGBM RMSE: {lgb_rmse:.3f}, XGB RMSE: {xgb_rmse:.3f}")


/tmp/ipython-input-4128088631.py:7: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  weather_df.fillna(method='ffill', inplace=True)
[I 2025-10-15 13:52:44,722] A new study created in memory with name: no-name-55b34eea-99aa-4e38-b121-bd50cd88b58c


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.060120 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5039
[LightGBM] [Info] Number of data points in the train set: 120000, number of used features: 47
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score -0.194694
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with po

[I 2025-10-15 13:52:50,751] Trial 0 finished with value: 0.6789914229889819 and parameters: {'n_estimators': 1054, 'learning_rate': 0.22921357096402842, 'num_leaves': 245, 'max_depth': 4, 'min_child_samples': 5, 'subsample': 0.9622098639357802, 'colsample_bytree': 0.8619240745014032}. Best is trial 0 with value: 0.6789914229889819.


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.058547 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5039
[LightGBM] [Info] Number of data points in the train set: 120000, number of used features: 47
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score -0.194694
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with po

[I 2025-10-15 13:53:00,073] Trial 1 finished with value: 0.6822567608514288 and parameters: {'n_estimators': 1635, 'learning_rate': 0.15306142057089406, 'num_leaves': 288, 'max_depth': 8, 'min_child_samples': 21, 'subsample': 0.6174946026886398, 'colsample_bytree': 0.6308398378626963}. Best is trial 0 with value: 0.6789914229889819.


...

[I 2025-10-15 13:59:27,769] Trial 48 finished with value: 0.6818539749804945 and parameters: {'n_estimators': 750, 'learning_rate': 0.2401300512529066, 'num_leaves': 220, 'max_depth': 8, 'min_child_samples': 10, 'subsample': 0.9180405572211483, 'colsample_bytree': 0.8470291081800144}. Best is trial 31 with value: 0.6780988815978101.


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.060002 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5039
[LightGBM] [Info] Number of data points in the train set: 120000, number of used features: 47
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score -0.194694
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with po

[I 2025-10-15 13:59:35,689] Trial 49 finished with value: 0.678369842163281 and parameters: {'n_estimators': 1662, 'learning_rate': 0.27293536649585676, 'num_leaves': 141, 'max_depth': 4, 'min_child_samples': 22, 'subsample': 0.651733808229694, 'colsample_bytree': 0.986094694015988}. Best is trial 31 with value: 0.6780988815978101.
2025/10/15 13:59:35 INFO mlflow.tracking.fluent: Experiment with name 'Ride Surge Prediction' does not exist. Creating a new experiment.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.069746 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5039
[LightGBM] [Info] Number of data points in the train set: 120000, number of used features: 47
[LightGBM] [Info] Start training from score -0.001115
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positiv

2025/10/15 14:00:06 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/15 14:00:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
[I 2025-10-15 14:02:00,410] A new study created in memory with name: no-name-be353b65-461d-4dd7-a6df-f1ae81bd9f94
[I 2025-10-15 14:02:34,677] Trial 0 finished with value: 0.7181726895498779 and parameters: {'n_estimators': 1636, 'learning_rate': 0.26267811919320977, 'max_depth': 7, 'subsample': 0.8313621531731299, 'colsample_bytree': 0.9190105445949637, 'min_child_weight': 1, 'gamma': 0.4455215672692948}. Best is trial 0 with value: 0.7181726895498779.
[I 2025-10-15 14:03:33,379] Trial 1 finished with value: 0.6662768253830902 and parameters: {'n_estimators': 2405, 'learning_rate': 0.04307019971696408, 'max_depth': 4, 'subsample': 0.8161834228348339, 'colsample_bytree': 0.6509755143876106

In [10]:
!zip -r ride_surge_project.zip /content
from google.colab import files
files.download("ride_surge_project.zip")

  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/active_config (stored 0%)
  adding: content/.config/.last_update_check.json (deflated 22%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2025.10.13/ (stored 0%)
  adding: content/.config/logs/2025.10.13/13.43.26.183753.log (deflated 57%)
  adding: content/.config/logs/2025.10.13/13.43.04.370331.log (deflated 58%)
  adding: content/.config/logs/2025.10.13/13.43.17.820264.log (deflated 58%)
  adding: content/.config/logs/2025.10.13/13.42.34.331861.log (deflated 92%)
  adding: content/.config/logs/2025.10.13/13.43.26.889435.log (deflated 56%)
  adding: content/.config/logs/2025.10.13/13.43.12.780756.log (deflated 86%)
  adding: content/.config/config_sentinel (stored 0%)
  adding: content/.config/default_configs.db (deflated 98%)
  adding: content/.config/gce (stored 0%)
  adding: content/.config/configurations/ (stored 0%)
  adding: content/.config/configurations/c

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>